# Market PlantoActual

## 1. Read data from MySQL
### 1.1. Import important packages

In [3]:
import pymysql
import pandas as pd
import decimal

### 1.2. Connect to MySQL database reconai

In [4]:
mydb = pymysql.connect(host='localhost',
                             user='root',
                             password='peacebro',database='reconai')
print(mydb)

### 1.3. Read data from database

In [5]:
sql_merged_market_sales='SELECT * FROM `merged_market_sales`;'
sql_marketplan='SELECT * FROM `marketplan`'

In [6]:
df_merged = pd.read_sql(sql_merged_market_sales, mydb)
df_merged.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Sales
0,XYZ Manufacturing,North,Mass,SuperMart,North,2017,1,Widgets,"10,00,000.00"
1,XYZ Manufacturing,East,Mass,SuperMart,East,2017,1,Widgets,"30,00,000.00"
2,XYZ Manufacturing,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,"7,50,000.00"
3,XYZ Manufacturing,South,Mass,SuperMart,Deep South,2017,1,Widgets,"6,00,000.00"
4,XYZ Manufacturing,West,Mass,SuperMart,West,2017,1,Widgets,"42,50,000.00"


In [8]:
df_marketplan = pd.read_sql(sql_marketplan, mydb)
df_marketplan.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Sales
0,XYZ Manufacturing,North,Mass,SuperMart,North,2017,1,Widgets,"10,22,500.00"
1,XYZ Manufacturing,East,Mass,SuperMart,East,2017,1,Widgets,"28,66,200.00"
2,XYZ Manufacturing,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,"7,42,950.00"
3,XYZ Manufacturing,South,Mass,SuperMart,Deep South,2017,1,Widgets,"6,24,180.00"
4,XYZ Manufacturing,West,Mass,SuperMart,West,2017,1,Widgets,"44,54,850.00"


## 2. Editing Dataframe for merge
### 2.1. Convert Sales column to float type

In [9]:
def sales_product(df):
    #df['date'] = pd.to_datetime(df.assign(day=1, month=1)[['Year', 'month', 'day']])+pd.to_timedelta(df.Week*7, unit='days')
    df['Sales'] = df['Sales'].str.replace(',', '')
    df['Sales'] = df['Sales'].astype(float)
sales_product(df_merged)
sales_product(df_marketplan)

### 2.2. Summing Gulf Coast and Deep South rows together

In [11]:
def merge_south(df):
    for year in [2017,2018]:
        for week in range(1,53):
            for cpg in ['XYZ Manufacturing','AAA Industries']:
                ex=df[(df['Retailer']=='SuperMart') & (df['CPG Region']=='South') & 
                    (df['Year']==year) & (df['Week']==week)
                                         & (df['CPG']==cpg)].sum()
                ex['Sales']=round(ex['Sales'], 2)
                df.loc[(df['Retailer']=='SuperMart') & (df['CPG Region']=='South') & 
                    (df['Year']==year) & (df['Week']==week) & 
                    (df['Category']=='Widgets')& (df['CPG']==cpg),
                    'Sales'] = ex['Sales']
    return df
df_merged=merge_south(df_merged)
df_merged.drop_duplicates(subset =["CPG Region","Retailer","Sales","Year","Week","CPG"], 
                     keep = "first", inplace = True)
df_marketplan=merge_south(df_marketplan)
df_marketplan.drop_duplicates(subset =["CPG Region","Retailer","Sales","Year","Week","CPG"], 
                     keep = "first", inplace = True)

### 2.3. Summing XYZ Manufacturing and AAA Industries rows together

In [13]:
def merge_cpg(df):
    for year in [2017,2018]:
        for week in range(1,53):
            for ret in ['SuperMart','BigCo','Rx']:
                for reg in ['North','East','South','West']:
                    ex=df[(df['Retailer']==ret) & (df['CPG Region']==reg) & 
                        (df['Year']==year) & (df['Week']==week)].sum()
                    ex['Sales']=round(ex['Sales'], 2)
                    df.loc[(df['Retailer']==ret) & (df['CPG Region']==reg) & 
                    (df['Year']==year) & (df['Week']==week) & 
                    (df['Category']=='Widgets'),
                    'Sales'] = ex['Sales']
    return df
df_merged=merge_cpg(df_merged)
df_merged.drop_duplicates(subset =["CPG Region","Retailer","Sales","Year","Week"], 
                     keep = "first", inplace = True)
df_marketplan=merge_cpg(df_marketplan)
df_marketplan.drop_duplicates(subset =["CPG Region","Retailer","Sales","Year","Week"], 
                     keep = "first", inplace = True)

                    CPG CPG Region Channel   Retailer Retailer Region  Year  \
0     XYZ Manufacturing      North    Mass  SuperMart           North  2017   
1     XYZ Manufacturing       East    Mass  SuperMart            East  2017   
2     XYZ Manufacturing      South    Mass  SuperMart      Gulf Coast  2017   
4     XYZ Manufacturing       West    Mass  SuperMart            West  2017   
9     XYZ Manufacturing      North    Mass      BigCo           North  2017   
10    XYZ Manufacturing       East    Mass      BigCo            East  2017   
11    XYZ Manufacturing      South    Mass      BigCo           South  2017   
12    XYZ Manufacturing       West    Mass      BigCo            West  2017   
17    XYZ Manufacturing      North    Drug         Rx           North  2017   
18    XYZ Manufacturing       East    Drug         Rx            East  2017   
19    XYZ Manufacturing      South    Drug         Rx           South  2017   
20    XYZ Manufacturing       West    Drug         R

In [10]:
df_merged.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Sales
0,XYZ Manufacturing,North,Mass,SuperMart,North,2017,1,Widgets,1000000.0
1,XYZ Manufacturing,East,Mass,SuperMart,East,2017,1,Widgets,3000000.0
2,XYZ Manufacturing,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,750000.0
3,XYZ Manufacturing,South,Mass,SuperMart,Deep South,2017,1,Widgets,600000.0
4,XYZ Manufacturing,West,Mass,SuperMart,West,2017,1,Widgets,4250000.0


## 3. Merge dataframes
### 3.1. Rename Sales column in both the dataframes

In [14]:
df_merged=df_merged.rename(columns={'Sales': 'Actual numbers'})

In [15]:
df_marketplan=df_marketplan.rename(columns={'Sales': 'Plan numbers'})
df_marketplan.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Plan numbers
0,XYZ Manufacturing,North,Mass,SuperMart,North,2017,1,Widgets,1022500.0
1,XYZ Manufacturing,East,Mass,SuperMart,East,2017,1,Widgets,4411650.0
2,XYZ Manufacturing,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,4912716.0
4,XYZ Manufacturing,West,Mass,SuperMart,West,2017,1,Widgets,7351370.0
9,XYZ Manufacturing,North,Mass,BigCo,North,2017,1,Widgets,7966500.0


### 3.2. Reset the index of the dataframes for merge

In [16]:
df_merged=df_merged.reset_index(drop=True)
df_marketplan=df_marketplan.reset_index(drop=True)
df_merged.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Actual numbers
0,XYZ Manufacturing,North,Mass,SuperMart,North,2017,1,Widgets,1000000.0
1,XYZ Manufacturing,East,Mass,SuperMart,East,2017,1,Widgets,4500000.0
2,XYZ Manufacturing,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,4920000.0
3,XYZ Manufacturing,West,Mass,SuperMart,West,2017,1,Widgets,7150000.0
4,XYZ Manufacturing,North,Mass,BigCo,North,2017,1,Widgets,8150000.0


### 3.3. Sales conversion
#### 3.3.1. Function to convert value to indian currency

In [17]:
def currency_in_indian_format(n):
    """ Convert a number (int / float) into indian formatting style """
    d = decimal.Decimal(str(n))

    if d.as_tuple().exponent < -2:
        s = str(n)
    else:
        s = '{0:.2f}'.format(n)

    l = len(s)
    i = l - 1

    res, flag, k = '', 0, 0
    while i >= 0:
        if flag == 0:
            res += s[i]
            if s[i] == '.':
                flag = 1
        elif flag == 1:
            k += 1
            res += s[i]
            if k == 3 and i - 1 >= 0:
                res += ','
                flag = 2
                k = 0
        else:
            k += 1
            res += s[i]
            if k == 2 and i - 1 >= 0:
                res += ','
                flag = 2
                k = 0
        i -= 1

    return res[::-1]

#### 3.3.2. Converting Actual numbers column to indian currency

In [18]:
for i in range (0,len(df_merged.index)):
    a=df_merged['Actual numbers'][i]
    b=currency_in_indian_format(a)
    df_merged['Actual numbers'][i]=b

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### 3.3.3. Converting Plan numbers column to indian currency

In [19]:
for i in range (0,len(df_marketplan.index)):
    a=df_marketplan['Plan numbers'][i]
    b=currency_in_indian_format(a)
    df_marketplan['Plan numbers'][i]=b

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### 3.4. Merge both dataframes

In [20]:
df_plantoactual_trial=pd.merge(df_marketplan, df_merged, on=("Retailer Region","CPG","CPG Region","Channel","Year","Week",
                                                    "Category","Retailer"))
df_plantoactual_trial.head()

,CPG,CPG Region,Channel,Retailer,Retailer Region,Year,Week,Category,Plan numbers,Actual numbers
0,XYZ Manufacturing,North,Mass,SuperMart,North,2017,1,Widgets,"10,22,500.00","10,00,000.00"
1,XYZ Manufacturing,East,Mass,SuperMart,East,2017,1,Widgets,"44,11,650.00","45,00,000.00"
2,XYZ Manufacturing,South,Mass,SuperMart,Gulf Coast,2017,1,Widgets,"49,12,716.00","49,20,000.00"
3,XYZ Manufacturing,West,Mass,SuperMart,West,2017,1,Widgets,"73,51,370.00","71,50,000.00"
4,XYZ Manufacturing,North,Mass,BigCo,North,2017,1,Widgets,"79,66,500.00","81,50,000.00"


In [21]:
df_plantoactual=pd.DataFrame(df_plantoactual_trial)
df_plantoactual=df_plantoactual.drop(['CPG','Channel','Retailer Region'],axis=1)
df_plantoactual.head()

,CPG Region,Retailer,Year,Week,Category,Plan numbers,Actual numbers
0,North,SuperMart,2017,1,Widgets,"10,22,500.00","10,00,000.00"
1,East,SuperMart,2017,1,Widgets,"44,11,650.00","45,00,000.00"
2,South,SuperMart,2017,1,Widgets,"49,12,716.00","49,20,000.00"
3,West,SuperMart,2017,1,Widgets,"73,51,370.00","71,50,000.00"
4,North,BigCo,2017,1,Widgets,"79,66,500.00","81,50,000.00"


## 4. Export as csv

In [22]:
df_plantoactual.to_csv("marketplantoactual.csv",index = False)